In [146]:
import os

import yfinance as yf
import requests
from datetime import datetime, timedelta
import time
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf


In [147]:
def get_time_stamp():
    return datetime.now().strftime('%Y%m%d%H%M%S')

In [148]:
directory = 'idx-' + get_time_stamp()
os.makedirs(directory, exist_ok=True)

In [149]:
def get_btc_idr(time_from, time_to) :
    pair = 'BTCIDR'
    time_frame = '1D'
    return requests.get('https://indodax.com/tradingview/history_v2?from=' + str(time_from) + '&symbol=' + pair +'&tf=' + time_frame + '&to=' + str(time_to)).json()

In [150]:
now = datetime.now()
past = now - timedelta(days=5*365)
time_from_s = int(time.mktime(past.timetuple()))
time_to_s = int(time.mktime(now.timetuple()))
time_from_date = past.strftime('%Y-%m-%d')
time_to_date = now.strftime('%Y-%m-%d')

In [151]:
def get_df_btc_idr(time_from, time_to) :
    btc_idr_daily = get_btc_idr(time_from, time_to)
    df_daily = pd.DataFrame(btc_idr_daily)
    if df_daily.empty :
        return df_daily
    df_daily['Date'] = pd.to_datetime(df_daily['Time'], unit='s')
    df_daily = df_daily.sort_values(by='Date')
    df_daily.set_index('Date', inplace=True)
    df_weekly = df_daily.resample('W-FRI').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })
    df_weekly.reset_index(inplace=True)
    return df_weekly

In [152]:
df_btc_idr = get_df_btc_idr(time_from_s, time_to_s)
print(df_btc_idr)

          Date        Open        High         Low       Close  \
0   2020-04-17   112275000   112532000   104199000   109858000   
1   2020-04-24   109857000   116275000   107406000   114739000   
2   2020-05-01   114740000   140600000   114387000   131352000   
3   2020-05-08   131401000   150000000   129261000   146499000   
4   2020-05-15   146498000   148071000   127517000   138432000   
..         ...         ...         ...         ...         ...   
257 2025-03-21  1375167000  1443105000  1345199000  1397882000   
258 2025-03-28  1397882000  1470000000  1390999000  1414996000   
259 2025-04-04  1414996000  1459452000  1359998000  1394431000   
260 2025-04-11  1394431000  1414696000  1280000000  1399995000   
261 2025-04-18  1399995000  1413000000  1394470000  1411999000   

                                                Volume  
0    625.31555025321.57136447350.45308541514.575318...  
1    200.57526728177.70041011320.31939571278.938527...  
2    187.16969307198.85135338343.204

In [153]:
stock_list_excel = pd.read_excel('Stock List.xlsx')
stock_list = stock_list_excel['Code']

In [154]:
# ticker_symbol = 'BREN.JK'

In [155]:
def get_df_stock_idr(ticker, time_from, time_to) :
    stock_daily = yf.download(ticker + '.JK', start=time_from, end=time_to, interval='1d', multi_level_index=False)
    stock_weekly = stock_daily.resample('W-FRI').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })
    stock_weekly.dropna(inplace=True)
    stock_weekly.reset_index(inplace=True)
    return stock_weekly

In [156]:
def calculate_change_price(df) :
    start = df['Close'].iloc[0]
    end = df['Close'].iloc[-1]
    percent = ((end-start)/start)*100
    return percent

In [157]:
def show_chart_change(df, chart_title) :
    width_ratio = 0.2
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='Change = 0%')
    plt.title(chart_title)
    plt.xlabel('Ticker')
    plt.ylabel('Perubahan (%)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, chart_title + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [158]:
def get_history_btc_pair(stock_ticker, df_stock_idr_weekly, df_btc_idr_weekly) :
    ticker_name_low = stock_ticker.lower()
    df_merged = pd.merge(df_stock_idr_weekly, df_btc_idr_weekly, on='Date', suffixes=('_'+ticker_name_low, '_btc'))
    df_merged['Open_' + ticker_name_low +'_btc'] = df_merged['Open_' + ticker_name_low] / df_merged['Open_btc']
    df_merged['High_' + ticker_name_low +'_btc'] = df_merged['High_' + ticker_name_low] / df_merged['High_btc']
    df_merged['Low_' + ticker_name_low + '_btc'] = df_merged['Low_' + ticker_name_low] / df_merged['Low_btc']
    df_merged['Close_'+ ticker_name_low +'_btc'] = df_merged['Close_' + ticker_name_low] / df_merged['Close_btc']
    df_merged['Volume_'+ ticker_name_low +'_btc'] = df_merged['Volume_' + ticker_name_low]

    df_final = df_merged[['Date', 'Open_' + ticker_name_low +'_btc', 'High_' + ticker_name_low +'_btc', 'Low_' + ticker_name_low + '_btc', 'Close_' + ticker_name_low +'_btc', 'Volume_' + ticker_name_low +'_btc']].copy()
    df_final.rename(columns={
        'Open_' + ticker_name_low +'_btc': 'Open',
        'High_' + ticker_name_low +'_btc': 'High',
        'Low_' + ticker_name_low + '_btc': 'Low',
        'Close_' + ticker_name_low +'_btc': 'Close',
        'Volume_' + ticker_name_low +'_btc': 'Volume'
    }, inplace=True)

    return df_final

In [159]:
# ticker_bbca = 'BBCA'
# df_bbca_idr = get_df_stock_idr(ticker_bbca, time_from_date, time_to_date)
# print(df_bbca_idr)
# df_bbca_btc = get_history_btc_pair(ticker_bbca, df_bbca_idr, df_btc_idr)
# print(df_bbca_btc)

In [160]:
def show_candlestick(ticker, df_stock_btc):
    df_stock_btc.set_index('Date', inplace=True, drop=False)
    mpf.plot(df_stock_btc,
             type='candle',
             style='binance',
             volume=True,
             title=ticker+' Weekly Pair BTC',
             ylabel='Harga (BTC)',
             savefig=dict(fname=os.path.join(directory, 'Candlestick-' + ticker + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight'))

In [161]:
def show_charts(tickers):
    change_3M_list = []
    change_6M_list = []
    change_1Y_list = []

    for ticker in tickers:
        print('Processing ' + ticker + '...')
        df_stock_idr = get_df_stock_idr(ticker, time_from_date, time_to_date)
        df_stock_btc = get_history_btc_pair(ticker, df_stock_idr, df_btc_idr)

        show_candlestick(ticker, df_stock_btc)

        three_months_ago = now - timedelta(days=90)
        six_months_ago = now - timedelta(days=180)
        years_ago = now - timedelta(days=365)

        df_three_months_ago = df_stock_btc[df_stock_btc['Date'] >= three_months_ago]
        df_six_months_ago = df_stock_btc[df_stock_btc['Date'] >= six_months_ago]
        df_years_ago = df_stock_btc[df_stock_btc['Date'] >= years_ago]

        latest_change_3M = calculate_change_price(df_three_months_ago)
        latest_change_6M = calculate_change_price(df_six_months_ago)
        latest_change_1Y = calculate_change_price(df_years_ago)

        change_3M_list.append({'Label': ticker, 'Data': latest_change_3M})
        change_6M_list.append({'Label': ticker, 'Data': latest_change_6M})
        change_1Y_list.append({'Label': ticker, 'Data': latest_change_1Y})

    change_3M_list = sorted(change_3M_list, key=lambda x: x['Data'])
    change_3M_list_df = pd.DataFrame(change_3M_list)
    change_6M_list = sorted(change_6M_list, key=lambda x: x['Data'])
    change_6M_list_df = pd.DataFrame(change_6M_list)
    change_1Y_list = sorted(change_1Y_list, key=lambda x: x['Data'])
    change_1Y_list_df = pd.DataFrame(change_1Y_list)

    show_chart_change(change_3M_list_df, 'Perubahan Harga 3 Bulan Terakhir Pair BTC')
    show_chart_change(change_6M_list_df, 'Perubahan Harga 6 Bulan Terakhir Pair BTC')
    show_chart_change(change_1Y_list_df, 'Perubahan Harga 1 Tahun Terakhir Pair BTC')

    excel_file = os.path.join(directory, 'price_idx-' + get_time_stamp() + '.xlsx')
    with pd.ExcelWriter(excel_file) as writer:
        change_3M_list_df.to_excel(writer, sheet_name='Change 3M', index=False)
        change_6M_list_df.to_excel(writer, sheet_name='Change 6M', index=False)
        change_1Y_list_df.to_excel(writer, sheet_name='Change 1Y', index=False)


In [162]:
show_charts(stock_list)

[*********************100%***********************]  1 of 1 completed

Processing AALI...


Processing ABBA...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Processing ABDA...


...

Processing EURO...



[*********************100%***********************]  1 of 1 completed

Processing KLIN...



[*********************100%***********************]  1 of 1 completed

Processing TOOL...
